In [1]:
# P1, Initialising
#various imported libraries, some are unused
#filenames may be irrelevant depending on who is accessing or what is available
import matplotlib.transforms as mtransforms
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
import numpy as np
import pickle
import os
from scipy.optimize import curve_fit
from astropy.io import fits
import pandas as pd
from astropy.utils.data import download_file
from astropy.table import Table
import statistics
from scipy.stats import powerlaw
from scipy.stats import iqr
import time

In [2]:
#FITS file reader function 
def rSDSSFITS(FITSfile):
    with fits.open(FITSfile) as hdulist:
        #allows for easy access of different headers from the FITS file. For DR14, the lite versions were sufficient. 
        (h0,d0,h1,d1,d2,h2)=(hdulist[0].header,hdulist[0].data,hdulist[1].header,hdulist[1].data,hdulist[2].data,hdulist[2].header)
        #flux array in angstrom
        lw=d1['loglam']
        #wavelength array in angstrom
        wl=np.power(10.0,lw)
        #the inverse variance array which corresponds to each wavelength value (for each pixel)
        invaris=np.array(d1['ivar'])
        #assignszeroes in ivar array as -inf
        sig=np.empty((len(invaris)))
        for j in range(0,len(invaris)):
            sig[j]=np.reciprocal(np.sqrt(invaris[j])) if (invaris[j]>0.0) else -np.inf
    #Returned variables 
    return {'wl':wl,'sig':sig,'lw':lw,'fl':d1['flux'],'er':d1[2],'invaris':invaris,'h0':h0,'d0':d0,'d1':d1,'h1':h1,'h2':h2,'d2':d2}


In [19]:
#imports input table of observations for processing
#accesses the input table and specifes the filetype 
dat = Table.read('enter table name', format='csv')
df = dat.to_pandas()
#displays the input table
df

,plate,plate mjd fiber_1,mjd,fiberid_1,run2d,specobj_id,ra,dec,z,zerr,...,Z_CIII,Z_MGII,Plate_2,Fiber,MJD_2,RA_2,DEC_2,Zem,flg_BAL,Plate-MJD-Fiber
0,5769,5769 56246 963,56246,963,'v5_10_0','6495581374791000064','09:10:09.59','+20:01:37.41',5.97305,0.001247,...,3.667171,-1.000000,5769,963,56246,137.5399,20.0271,5.973559,0,5769 56246 963
1,4216,4216 55477 19,55477,19,'v5_10_0','4746799321818570752','00:05:52.36','-00:06:56.31',5.86176,0.000514,...,-1.000000,-1.000000,4216,19,55477,1.4681,-0.1155,5.863910,0,4216 55477 19
2,6424,6424 56272 731,56272,731,'v5_10_0','7232982042534715392','10:22:10.05','+22:52:25.44',5.48276,0.000324,...,-1.000000,-1.000000,6424,731,56272,155.5419,22.8737,5.482432,0,6424 56272 731
3,4389,4389 55539 679,55539,679,'v5_10_0','4941761426161115136','02:31:37.65','-07:28:54.47',5.42329,0.000238,...,-1.000000,-1.000000,4389,679,55539,37.9069,-7.4818,5.411926,0,4389 55539 679
4,4834,4834 55685 566,55685,566,'v5_10_0','5442755825952071680','12:33:33.48','+06:22:34.17',5.31090,0.000356,...,-1.000000,-1.000000,4834,566,55685,188.3895,6.3762,5.333209,0,4834 55685 566
5,6485,6485 56342 582,56342,582,'v5_10_0','7301620981218385920','12:49:36.60','+33:30:30.17',5.27174,0.000712,...,-1.000000,-1.000000,6485,582,56342,192.4025,33.5084,5.291029,0,6485 56342 582
6,4716,4716 55738 2,55738,2,'v5_10_0','5309744606694318080','15:23:45.69','+33:47:59.40',5.37546,0.001008,...,-1.000000,-1.000000,4716,2,55738,230.9404,33.8000,5.279680,0,4716 55738 2
7,6016,6016 56073 340,56073,340,'v5_10_0','6773507399942643712','14:37:51.82','+23:23:13.35',5.32249,0.000238,...,-1.000000,-1.000000,6016,340,56073,219.4659,23.3870,5.279321,0,6016 56073 340
8,5370,5370 56003 618,56003,618,'v5_10_0','6046252475006033920','11:22:50.04','+13:42:36.43',5.19745,0.000613,...,-1.000000,-1.000000,5370,618,56003,170.7085,13.7101,5.272259,0,5370 56003 618
9,6464,6464 56309 680,56309,680,'v5_10_0','7278004020655923200','10:26:23.63','+25:42:59.45',5.27788,0.000680,...,-1.000000,-1.000000,6464,680,56309,156.5985,25.7165,5.255236,0,6464 56309 680


In [86]:
#defining the spectral regions, medians and the calculated minimum expectred value for pixels in each region
spre=np.array([[1276,1292],[1314,1328],[1345,1362],[1435,1465],[1684,1700]])
medspre=(1284,1321,1353.5,1450,1692)
minval=(27,23,27,44,20)
#defines the powerlaw functin for which the curve is fit
def func_powerlaw(lam, m, c):
    return c*lam**m
#a simple timing function
start = time.time()
#initialises lists that values for each spectrum will append to:
#spectral index or Beta
Beta=[]
#The error in Beta
Berrm=[]
#The Plate-MJD-Fiber identifier
Bname=[]
#the number of pixels present in a spectral region,fval is explained below
lenfval=[]
#the parent loop that iterates over the input list, 20298 should be replaced with the length of the list
for t in range(0,20298):
    #a simple progress indicator
    if t%1000==0:
        print(t)
    #allows the files to be iteratively searched and referenced    
    FITSfile='spec'+'-'+str(int(df.iloc[t]['plate']))+'-'+str(int(df.iloc[t]['mjd']))+'-'+str(int(df.iloc[t]['fiberid_1'])).zfill(4)+'.fits'
    #defines the function above
    s=rSDSSFITS(FITSfile)
    #defines the sigma,ivar and redshift value for each spectra from the header of the respective fits file.
    siggie=s['sig']
    invar=s['invaris']
    zuse=df.iloc[t]['Zem']
    #corrects the redshift value for the observed frame
    z1=zuse+1
    ospre=spre*z1
    #defines the wavelength and flux array respectively
    x=s['wl']
    y=np.array(s['fl'])
    #initialises the 'error flag' each loop
    AA=[]
    
    #fval are the flux values that correspond to wavelengths in the spectral regions
    fval=[]
    for i in range(0,5):
            g1=y[(x>=ospre[i][0]) & (x<=ospre[i][1])]
            fval.append(g1) 
            g456=len(fval[i])
            lenfval.append(g456)
    #simultaneous determination of median flux values and the iqr for each region.        
    iqrval=[]
    fmed=[] 
    #print(len(fval[0]),len(fval[1]),len(fval[2]),len(fval[3]),len(fval[4]),z1)
    ##checks that each region has a minimum of half the expected pixels, if it does then it determines median and iqr
    ##if not then it assigns AA to be 1, which is used later as an error flag.
    for i in range(0,5):
        if fval[i].size<=minval[i]:
            AA=1
            break
        else:   
            iq2=np.percentile(fval[i], [25, 50, 75])
            g2=iq2[2]-iq2[0]
            iqrval.append(g2)
            fmed.append(iq2[1])
    #produces the corresponding ivar values for wavelength values in spectral regions, not actually neccesary.        
    inval=[]
    for i in range(0,5):
            g5=invar[(x>=ospre[i][0]) & (x<=ospre[i][1])]
            inval.append(g5)   
    #Produces the corresponding sigma values for each wavelength value in spectral regions        
    sigval=[]
    for i in range(0,5):
            g5=siggie[(x>=ospre[i][0]) & (x<=ospre[i][1])]
            sigval.append(g5)  
    #converts the rest median values to whatever the observed redshift would be, AA condition but it isn't meaningful         
    medspreg=[]
    for i in range(0,5):
        if AA==1:
            break
        else:   
            g0=medspre[i]*z1
            medspreg.append(g0)   
    #produces a list of flux values that are 'good' based solely on the corresponding sigma value  
    ffval=[]
    for i in range (0,5):
        if AA==1:
            break
        else:
            g6=np.array(fval[i])[sigval[i]>0]
            ffval.append(g6)
    #AA condition, if it has been set to 1, set check to (0,0) which will later fail the bad data in region test, 
    #otherwise the fraction of good fval values against the number of sigval values (could be fval, same length)
    check=[]    
    for i in range (0,5):
        if AA==1:
            check=(0,0)
            break
        else:
            g17=len(ffval[i])/len(sigval[i])
            check.append(g17)
    # where Mean essentially acts as a flag for no data or bad data. 
    #if successful then it performs the calculations of Beta determination as normal
    #if not then -np.inf is appended
    if all(g > .5 for g in check):
        (X,Y)=(medspreg,fmed)
        popt, pcov = curve_fit(func_powerlaw,X,Y,sigma=iqrval,maxfev=130000)
        #The three quantities below are the final result assuming the checks are passed, the name value is used for 
        #list syncronisation when used in a program like topcat
        Beta.append(popt[0])
        Berrm.append(np.sqrt(np.diag(pcov))[0])
        Bname.append(df.iloc[t]['Plate-MJD-Fiber'])
    #if the check is failed then the two values are stored as -np.inf    
    else:
        Beta.append(-np.inf)
        Berrm.append(-np.inf)
        Bname.append(df.iloc[t]['Plate-MJD-Fiber'])
print(time.time() - start) 
#pickle is a useful utility that preserves a variable as a file that can be later opened, review usage on stack overflow
f = open('Beta1.pckl', 'wb')
pickle.dump(Beta, f)
pickle.dump(Berrm, f) 
pickle.dump(Bname, f)
f.close()

0
19.27848196029663
